<a href="https://colab.research.google.com/github/chunjie-sam-liu/TEP-prognosis/blob/main/test/pytorch_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
VERSION = "nightly" #@param ["20200220","nightly", "xrt==1.15.0"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  50156      0 --:--:-- --:--:-- --:--:-- 50156
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 3.0MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
Done updating TPU runtime
  Successfully uninstalled torchvision-0.8.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 94.3 MiB/ 94.3 MiB]                                                
Operation completed over 1 objects/94.3 MiB.                                     
Copying gs://

In [6]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 77kB/s 
  Found existing installation: torch-xla 1.6+ee12b88
    Uninstalling torch-xla-1.6+ee12b88:
      Successfully uninstalled torch-xla-1.6+ee12b88


In [7]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [8]:
# Create a random tensor on xla:1 
dev = xm.xla_device()
t1 = torch.ones(3, 3, device=dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


In [9]:
second_dev = xm.xla_device(n = 2, devkind='TPU')
t2 = torch.zeros(3, 3, device=second_dev)
print(t2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='xla:2')


In [10]:
a = torch.randn(2, 2, device=dev)
b = torch.randn(2, 2, device=dev)
print(a + b)
print(b * 2)
print(torch.matmul(a, b))

tensor([[-0.3088,  0.6661],
        [ 1.3402, -0.4989]], device='xla:1')
tensor([[-1.2814,  3.1716],
        [ 2.1050, -2.6671]], device='xla:1')
tensor([[-1.1809,  1.7529],
        [ 0.6977, -0.6614]], device='xla:1')


In [12]:
filters = torch.randn(33, 16, 3, device=dev)
inputs = torch.randn(20, 16, 50, device=dev)
torch.nn.functional.conv1d(inputs, filters)

tensor([[[ 4.6650e+00,  2.4131e+00,  8.4184e+00,  ..., -3.4691e+00,
           1.3903e+00, -1.0927e+00],
         [ 3.4179e+00,  8.0244e+00, -9.9459e+00,  ..., -8.1479e+00,
          -3.1750e+00,  4.1194e+00],
         [ 1.3079e+01,  4.3412e-01,  4.8352e+00,  ..., -5.5864e+00,
          -5.9210e+00,  3.0204e+00],
         ...,
         [-9.0025e-01,  1.0062e+00, -7.9940e+00,  ..., -3.3179e-01,
           1.4145e+00, -1.6179e+00],
         [ 6.8360e+00, -7.3494e+00, -5.3362e+00,  ..., -2.9572e+00,
          -2.2959e-02,  3.9403e+00],
         [ 5.0153e+00,  8.0487e+00,  1.1326e+01,  ..., -1.4361e+00,
          -3.4222e+00, -1.2024e+01]],

        [[ 7.2372e+00,  4.9307e+00, -3.2422e-02,  ..., -1.2313e+00,
          -7.7229e+00, -2.7538e-01],
         [ 1.3342e+00, -3.3721e+00, -3.7326e+00,  ...,  1.9647e+00,
           2.9404e+00, -2.5306e+00],
         [-1.4254e+00, -1.2025e+01, -8.0462e+00,  ..., -9.1502e+00,
          -4.7130e+00, -8.2591e+00],
         ...,
         [ 1.4655e-01,  2

In [13]:
t_cpu = torch.rand(2, 2, device='cpu')
print(t_cpu)
t_tpu = t_cpu.to(dev)
print(t_tpu)
t_cpu_again = t_tpu.to('cpu')
print(t_cpu_again)

tensor([[0.7431, 0.7050],
        [0.8456, 0.2437]])
tensor([[0.7431, 0.7050],
        [0.8456, 0.2437]], device='xla:1')
tensor([[0.7431, 0.7050],
        [0.8456, 0.2437]])


In [14]:
fc = torch.nn.Linear(5, 2, bias=True)
fc = fc.to(dev)
features = torch.rand(3, 5, device=dev, requires_grad=True)
output = fc(features)
print(output)

tensor([[-0.1844,  0.2907],
        [-0.3993,  0.2969],
        [-0.0952,  0.4496]], device='xla:1', grad_fn=<AddmmBackward>)


In [15]:
output.backward(torch.ones_like(output))
print(fc.weight.grad)

tensor([[0.7920, 1.6270, 1.3105, 1.3555, 0.8315],
        [0.7920, 1.6270, 1.3105, 1.3555, 0.8315]], device='xla:1')


In [16]:
import torch.nn as nn
import torch.nn.functional as F

# Simple example network from 
# https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# Places network on the default TPU core
net = Net().to(dev)

# Creates random input on the default TPU core
input = torch.randn(1, 1, 32, 32, device=dev)

# Runs network
out = net(input)
print(out)

tensor([[ 0.1949, -0.0153, -0.0022,  0.0143, -0.1247,  0.0424, -0.1731, -0.0325,
          0.0929,  0.0782]], device='xla:1', grad_fn=<AddmmBackward>)
